In [18]:
# 1. Imports e configuração de caminho
import sys, os
import pandas as pd

# Adiciona src/ ao path
src_path = os.path.join(os.getcwd(), "src")
sys.path.append(src_path)

from fetchers import fetch_inmet_climate, fetch_ibge_population_density

# Cria pasta de saída
os.makedirs("data/processed", exist_ok=True)

In [19]:
# 2. Carrega base 
DENGUE_URL = (
    "https://raw.githubusercontent.com/"
    "valdineyatilio/ProjetoAplicado-IV/"
    "main/A1/DadosDengueCSV.csv"
)

df_dengue = pd.read_csv(DENGUE_URL)
print("Base de dengue carregada:", df_dengue.shape)

Base de dengue carregada: (812, 30)


In [20]:
print("Colunas disponíveis:", df_dengue.columns.tolist())

Colunas disponíveis: ['data_iniSE', 'SE', 'casos_est', 'casos_est_min', 'casos_est_max', 'casos', 'p_rt1', 'p_inc100k', 'Localidade_id', 'nivel', 'id', 'versao_modelo', 'tweet', 'Rt', 'pop', 'tempmin', 'umidmax', 'receptivo', 'transmissao', 'nivel_inc', 'umidmed', 'umidmin', 'tempmed', 'tempmax', 'casprov', 'casprov_est', 'casprov_est_min', 'casprov_est_max', 'casconf', 'notif_accum_year']


In [21]:
# 3. Coleta clima INMET 
stations = ["A507", "A518", "A541"]  

try:
    df_clima = fetch_inmet_climate(stations, "2015-01-01", "2025-12-31")
except RuntimeError:
    print("[WARN] Nenhuma estação INMET retornou dados válidos. Gerando dados simulados.")

    import numpy as np
    weeks = pd.date_range("2018-01-01", "2023-12-31", freq="W-MON")
    df_clima = pd.DataFrame({
        "datahora": weeks,
        "station": "SIMULADA",
        "year": weeks.isocalendar().year,
        "week": weeks.isocalendar().week,
        "t_mean": np.random.normal(loc=22, scale=3, size=len(weeks)),
        "precip_sum": np.random.gamma(shape=2, scale=10, size=len(weeks))
    })

# Salva clima
df_clima.to_csv("data/processed/clima_inmet.csv", index=False)
print("Clima salvo:", df_clima.shape)

[WARN] Estação A507 retornou status 404. Pulando.
[WARN] Estação A518 retornou status 404. Pulando.
[WARN] Estação A541 retornou status 404. Pulando.
[WARN] Nenhuma estação INMET retornou dados válidos. Gerando dados simulados.
Clima salvo: (313, 6)


In [22]:
# 4. Coleta densidade populacional via IBGE
mun_ids = df_dengue["Localidade_id"].unique().tolist()
df_pop = fetch_ibge_population_density(mun_ids, 2020)

# Salva população
df_pop.to_csv("data/processed/pop_density.csv", index=False)
print("População salva:", df_pop.shape)

[WARN] IBGE retornou status 500 para município 0. Pulando.


RuntimeError: Nenhum município IBGE retornou dados válidos.